In [20]:
import os

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display

import utils
import psycopg2

In [21]:
tracks = utils.load('data/fma_metadata/tracks.csv')
genres = utils.load('data/fma_metadata/genres.csv')
features = utils.load('data/fma_metadata/features.csv')
echonest = utils.load('data/fma_metadata/echonest.csv')

## Get Features

In [130]:
features_scaled = skl.preprocessing.StandardScaler().fit_transform(features)
scaled_df = pd.DataFrame(features_scaled, index=features.index, columns=features.columns)

In [131]:
pca = skl.decomposition.PCA(n_components=20)
pca_fit = pca.fit(scaled_df)
features_transformed = pca_fit.transform(features_scaled)
features_output = pd.DataFrame(features_transformed[:, :3], index=features.index)

## Database Insertion

In [146]:
conn = psycopg2.connect(database="musiclib", user='postgres', password='password', host='127.0.0.1', port= '5432')
cursor = conn.cursor()

In [133]:
genre_id2name = {}
for index, row in genres.iterrows():
    genre_id2name[index] = row['title']

In [134]:
stored_artist_ids = set()

In [135]:
def read_track(track_id, row):
    #insert artist if not in table
    artist_id = row['artist']['id']
    if artist_id not in stored_artist_ids:
        stored_artist_ids.add(artist_id)
        artist_loc = row['artist']['location']
        if pd.isnull(artist_loc):
            artist_loc = None 
        elif len(artist_loc) > 50:
            artist_loc = artist_loc[:45]
        artist_name = row['artist']['name']
        artist_bio = row['artist']['bio']
        if artist_bio == 'nan':
            artist_bio = None
        if artist_loc == 'nan':
            artist_loc = None

        insert_artist_sql = """
        INSERT INTO artist(id, name, bio, location) 
        VALUES(%s, %s, %s, %s);
        """
        try:
            #print("Inserting artist record")
            cursor.execute(insert_artist_sql, (artist_id,artist_name,artist_bio,artist_loc))
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
            conn.rollback()
            return
    
    #insert track
    track_title = row['track']['title']
    if pd.isnull(track_title):
        track_title = None
    elif len(track_title) > 100:
        track_title = track_title[:95]
    interest = row['track']['interest']
    listens = row['track']['listens']
    date_created = row['track']['date_created']
    duration = row['track']['duration']
    language = row['track']['language_code']
    if pd.isnull(language):
        language = None
    elif len(language) > 10:
        language = language[:9]
    features = features_output.loc[track_id]
    insert_track_sql = """
    INSERT INTO track(id, title, artist_id, interest, listens, date_created, duration, language, feature1, feature2, feature3) 
    VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    try:
        #print("Inserting track record")
        cursor.execute(insert_track_sql, (track_id, track_title, artist_id, interest, listens, date_created, duration, language, features[0], features[1], features[2]))
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        conn.rollback()
        return

    #insert genres
    genre_ids = row['track']['genres_all']
    insert_genre_sql = '''
    INSERT INTO genre(track_id, genre) 
    VALUES(%s, %s);
    '''
    for gid in genre_ids:
        try:
            #print("Inserting genre record")
            cursor.execute(insert_genre_sql, (track_id, genre_id2name[gid]))
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
            conn.rollback()
            return

In [136]:
num_tracks = tracks.shape[0]

In [137]:
num_rows = 0
for index, row in tracks.iterrows():
    read_track(index, row)
    num_rows += 1
    if num_rows % 1000 == 0:
        print("{} tracks out of {}".format(num_rows, num_tracks))

1000 tracks out of 106574
2000 tracks out of 106574
3000 tracks out of 106574
4000 tracks out of 106574
5000 tracks out of 106574
6000 tracks out of 106574
7000 tracks out of 106574
8000 tracks out of 106574
9000 tracks out of 106574
10000 tracks out of 106574
11000 tracks out of 106574
12000 tracks out of 106574
13000 tracks out of 106574
14000 tracks out of 106574
15000 tracks out of 106574
16000 tracks out of 106574
17000 tracks out of 106574
18000 tracks out of 106574
19000 tracks out of 106574
20000 tracks out of 106574
21000 tracks out of 106574
22000 tracks out of 106574
23000 tracks out of 106574
24000 tracks out of 106574
25000 tracks out of 106574
26000 tracks out of 106574
27000 tracks out of 106574
28000 tracks out of 106574
29000 tracks out of 106574
30000 tracks out of 106574
31000 tracks out of 106574
32000 tracks out of 106574
33000 tracks out of 106574
34000 tracks out of 106574
35000 tracks out of 106574
36000 tracks out of 106574
37000 tracks out of 106574
38000 trac

## Load Audio Paths

In [147]:
gen_paths = 'data/fma_small/'
def fill(x):
    xs = str(x)
    while len(xs) < 3:
        xs = '0' + xs
    return xs
dirs = [fill(i) for i in range(156)]

In [148]:
insert_audio_sql = "INSERT INTO audio(track_id, file_path) VALUES(%s, %s);"
for d in dirs:
    audio_path = gen_paths + d + '/'
    dir_list = os.listdir(audio_path)
    for f in dir_list:
        track_id = int(f[:f.index('.')])
        full_relpath = audio_path + f
        try:
            cursor.execute(insert_audio_sql, (track_id, full_relpath))
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
            conn.rollback()
            break

In [149]:
conn.close()
cursor.close()